In [112]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import re

In [113]:
train_df = pd.read_csv("Divar.csv")

C:\Users\USER\AppData\Local\Temp\ipykernel_7928\960344712.py:1: DtypeWarning: Columns (11,27,29,53) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("Divar.csv")


In [162]:
df_subset = pd.concat([train_df.iloc[:, :5],train_df.iloc[:, 7:9],train_df.iloc[:, 58:]], axis=1)
df_geo = pd.read_csv("utm_coords1.csv")


In [163]:
df_subset["UTM_Easting"] = df_geo["UTM_Easting"]
df_subset["UTM_Northing"] = df_geo["UTM_Northing"]

df_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 12 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   Unnamed: 0          1000000 non-null  int64  
 1   cat2_slug           1000000 non-null  object 
 2   cat3_slug           999999 non-null   object 
 3   city_slug           999998 non-null   object 
 4   neighborhood_slug   437139 non-null   object 
 5   description         1000000 non-null  object 
 6   title               999946 non-null   object 
 7   location_latitude   655608 non-null   float64
 8   location_longitude  655608 non-null   float64
 9   location_radius     339699 non-null   float64
 10  UTM_Easting         1000000 non-null  float64
 11  UTM_Northing        1000000 non-null  float64
dtypes: float64(5), int64(1), object(6)
memory usage: 91.6+ MB


In [ ]:
from sklearn.neighbors import NearestNeighbors

def fill_neighborhood_with_utm(group, max_distance_m=2500):
    known = group.dropna(subset=["neighborhood_slug", "UTM_Easting", "UTM_Northing"])
    missing = group[group["neighborhood_slug"].isna() & 
                    group["UTM_Easting"].notna() & 
                    group["UTM_Northing"].notna()]
    
    if known.empty or missing.empty:
        return group
    
    # Fit NearestNeighbors
    nn = NearestNeighbors(n_neighbors=1, metric="euclidean")
    X_known = known[["UTM_Easting", "UTM_Northing"]].values
    nn.fit(X_known)

    X_missing = missing[["UTM_Easting", "UTM_Northing"]].values
    distances, indices = nn.kneighbors(X_missing)

    for i, idx in enumerate(missing.index):
        dist = distances[i][0]
        nearest_idx = known.index[indices[i][0]]
        if dist <= max_distance_m:  # threshold in meters
            group.loc[idx, "neighborhood_slug"] = known.loc[nearest_idx, "neighborhood_slug"]
    
    return group

In [165]:

df_subset = df_subset.groupby("city_slug", group_keys=False).apply(fill_neighborhood_with_utm, max_distance_m=2500)

C:\Users\USER\AppData\Local\Temp\ipykernel_7928\305110124.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_subset = df_subset.groupby("city_slug", group_keys=False).apply(fill_neighborhood_with_utm, max_distance_m=2500)


In [ ]:
from sklearn.neighbors import NearestNeighbors


def fill_neighborhood_by_location(group, max_distance_km=3.0):
    """
    Fills missing neighborhood_slug values using nearest known neighbor
    within the same city, based on lat/lng coordinates.
    
    max_distance_km: maximum distance (in km) allowed for neighbor assignment.
    """
    known = group.dropna(subset=["neighborhood_slug", "location_latitude", "location_longitude"])
    missing = group[group["neighborhood_slug"].isna() & group["location_latitude"].notna()]

    if known.empty or missing.empty:
        return group
    
    nn = NearestNeighbors(n_neighbors=1, metric="haversine")
    nn.fit(np.radians(known[["location_latitude", "location_longitude"]].values))
    
    distances, indices = nn.kneighbors(np.radians(missing[["location_latitude", "location_longitude"]].values))
    
    # Convert radian distance to kilometers (Earth radius ~ 6371 km)
    distances_km = distances.flatten() * 6371
    

    fill_values = []
    for i, idx in enumerate(missing.index):
        if distances_km[i] <= max_distance_km:
            fill_values.append(known.iloc[indices[i,0]]["neighborhood_slug"])
        else:
            fill_values.append(np.nan)  # too far, leave as NaN
    
    group.loc[missing.index, "neighborhood_slug"] = fill_values
    return group

In [166]:

df_subset = df_subset.groupby("city_slug", group_keys=False).apply(fill_neighborhood_by_location)

C:\Users\USER\AppData\Local\Temp\ipykernel_7928\2619927270.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_subset = df_subset.groupby("city_slug", group_keys=False).apply(fill_neighborhood_by_location)


In [167]:
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 999998 entries, 0 to 999999
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          999998 non-null  int64  
 1   cat2_slug           999998 non-null  object 
 2   cat3_slug           999997 non-null  object 
 3   city_slug           999998 non-null  object 
 4   neighborhood_slug   445203 non-null  object 
 5   description         999998 non-null  object 
 6   title               999944 non-null  object 
 7   location_latitude   655608 non-null  float64
 8   location_longitude  655608 non-null  float64
 9   location_radius     339699 non-null  float64
 10  UTM_Easting         999998 non-null  float64
 11  UTM_Northing        999998 non-null  float64
dtypes: float64(5), int64(1), object(6)
memory usage: 131.4+ MB


In [145]:
df_subset

,Unnamed: 0,cat2_slug,cat3_slug,city_slug,neighborhood_slug,description,title
0,0,temporary-rent,villa,karaj,mehrshahr,۵۰۰متر\n۲۰۰متر بنا دوبلکس\n۳خواب\nاستخر آبگرم ...,باغ ویلا اجاره روزانه استخر داخل لشکرآباد سهیلیه
1,1,residential-sell,apartment-sell,tehran,gholhak,دسترسی عالی به مترو و شریعتی \nمشاعات تمیز \nب...,۶۰ متر قلهک فول امکانات
2,2,residential-rent,apartment-rent,tehran,tohid,تخلیه پایان ماه,آپارتمان ۳ خوابه ۱۳۲ متر
3,3,commercial-rent,office-rent,tehran,elahiyeh,فرشته تاپ لوکیشن\n۹۰ متر موقعیت اداری\nیک اتاق...,فرشته ۹۰ متر دفتر کار مدرن موقعیت اداری
4,4,residential-sell,apartment-sell,mashhad,emamreza,هلدینگ ساختمانی اکبری\n\nهمراه شما هستیم برای ...,۱۱۵ متری/شمالی رو به آفتاب/اکبری
...,...,...,...,...,...,...,...
999995,999995,residential-sell,apartment-sell,kermanshah,NaN,~~~مشاورین املاک قبادی~~~\n■جنوبی تک واحدی\n■د...,آپارتمان ۱۸۰ متری وحدت غربی
999996,999996,residential-rent,apartment-rent,tehran,darya,نوساز \n\n تک واحدی\n\nشخصی ساز\n\nروف گا...,آپارتمان ۱۱۰ متری سعادت آباد دریا
999997,999997,residential-sell,house-villa-sell,yazd,NaN,سلام ودرود\nفروش منزل مسکونی واقع در خیابان ان...,منزل فروشی. خیابان انقلاب نرسیده به کارخانه...
999998,999998,temporary-rent,suite-apartment,bandar-anzali,NaN,سویت بدون خواب (روبه دریا و معمولی)\nسویت ۱خوا...,مجتمع ویلایی کنار ساحل پاسداران


In [168]:
nan_neighbor_mask = (~(df_subset.neighborhood_slug.isna()))

print(nan_neighbor_mask.sum())
# df_subset[nan_neighbor_mask].head(25)

445203


In [109]:
x = df_subset.loc[df_subset["Unnamed: 0"] == 22	, "description"].iloc[0]
y = df_subset.loc[df_subset["Unnamed: 0"] == 97	, "description"].iloc[0]

print(x)

# print(y)

زمین واقع در کلاته گیزی 
پنج کیلو متر بعد از ابراهیم اباد جاده اسفالت 
از لب جاده ۱۰۰۰ متر فاصله زمین 
زمین کاملا مسطح و صاف شده 
راه کشی شده قطعه مشخص 
اماده قطعه بندی 
چهارمیخ 
تمامی نامه ها تاییدیه ها سورا و دهیاری تقسیم نامه ها قولنامه ها موجود و مرتب 
زمین مزروعی 
معاوضه با اپارتمان الهیه امامیه و…


In [169]:
city_counts = df_subset.groupby("city_slug").size()
small_cities = city_counts[city_counts <= 1300].index

small_cities_rows = df_subset[df_subset["city_slug"].isin(small_cities)]
print(f"total rows = {len(small_cities_rows)}")

nan_neighbor_rows = df_subset[(df_subset["city_slug"].isin(small_cities)) & (df_subset.neighborhood_slug.isna())]

print(len(small_cities_rows))
print(len(nan_neighbor_rows))
print(len(small_cities))


total rows = 111465
111465
111465
313


In [170]:
nan_neighbor_small_cities = ((df_subset["city_slug"].isin(small_cities)) & (df_subset.neighborhood_slug.isna()))

df_subset.loc[nan_neighbor_small_cities, "neighborhood_slug"] = df_subset.loc[nan_neighbor_small_cities, "city_slug"]

In [171]:
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 999998 entries, 0 to 999999
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          999998 non-null  int64  
 1   cat2_slug           999998 non-null  object 
 2   cat3_slug           999997 non-null  object 
 3   city_slug           999998 non-null  object 
 4   neighborhood_slug   556668 non-null  object 
 5   description         999998 non-null  object 
 6   title               999944 non-null  object 
 7   location_latitude   655608 non-null  float64
 8   location_longitude  655608 non-null  float64
 9   location_radius     339699 non-null  float64
 10  UTM_Easting         999998 non-null  float64
 11  UTM_Northing        999998 non-null  float64
dtypes: float64(5), int64(1), object(6)
memory usage: 131.4+ MB


In [172]:
cleaned_data = df_subset.drop(["location_latitude", "location_longitude", "location_radius", "description", "title", "cat2_slug"], axis=1)
pd.DataFrame.to_csv(cleaned_data, "E:\AI_Quera\Real-estate_PricePrediction\processed data\cleaned_0_8.csv")

<>:2: SyntaxWarning: invalid escape sequence '\A'
<>:2: SyntaxWarning: invalid escape sequence '\A'
C:\Users\USER\AppData\Local\Temp\ipykernel_7928\996102480.py:2: SyntaxWarning: invalid escape sequence '\A'
  pd.DataFrame.to_csv(cleaned_data, "E:\AI_Quera\Real-estate_PricePrediction\processed data\cleaned_0_8.csv")


In [173]:
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 999998 entries, 0 to 999999
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          999998 non-null  int64  
 1   cat2_slug           999998 non-null  object 
 2   cat3_slug           999997 non-null  object 
 3   city_slug           999998 non-null  object 
 4   neighborhood_slug   556668 non-null  object 
 5   description         999998 non-null  object 
 6   title               999944 non-null  object 
 7   location_latitude   655608 non-null  float64
 8   location_longitude  655608 non-null  float64
 9   location_radius     339699 non-null  float64
 10  UTM_Easting         999998 non-null  float64
 11  UTM_Northing        999998 non-null  float64
dtypes: float64(5), int64(1), object(6)
memory usage: 131.4+ MB
